In [336]:
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from XBNet.training_utils import training,predict
from XBNet.models import XBNETClassifier
from XBNet.run import run_XBNET

import glob
from pandas.core.common import flatten
import random

In [337]:
# set path split1
split = 'split1'
path = "../5-fold_new/DATA"
train_path = "{}/{}/train/train_{}.csv".format(path,split,split)
test_path =  "{}/test.csv".format(path)
valid_path = "{}/{}/val/val_{}.csv".format(path,split,split)

In [338]:
print(train_path)
print(test_path)
print(valid_path)

../5-fold_new/DATA/split1/train/train_split1.csv
../5-fold_new/DATA/test.csv
../5-fold_new/DATA/split1/val/val_split1.csv


In [339]:
train_data = pd.read_csv(train_path)
print(train_data.shape)
x_train = train_data[train_data.columns[2:]]
print(x_train.shape)
y_train = train_data[train_data.columns[1]]
le = LabelEncoder()
y_train = np.array(le.fit_transform(y_train))
print(le.classes_)

(364, 32)
(364, 30)
['B' 'M']


In [340]:
# Load the validation set
val_data = pd.read_csv(valid_path)
print(val_data.shape)
x_val = val_data[val_data.columns[2:]]
print(x_val.shape)
y_val = val_data[val_data.columns[1]]
le = LabelEncoder()
y_val = np.array(le.fit_transform(y_val))
print(le.classes_)

In [341]:
train_data

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,879523,M,15.120,16.68,98.78,716.6,0.08876,0.09588,0.07550,0.04079,...,17.770,20.24,117.70,989.5,0.14910,0.33310,0.33270,0.12520,0.3415,0.09740
1,879804,B,9.876,17.27,62.92,295.4,0.10890,0.07232,0.01756,0.01952,...,10.420,23.22,67.08,331.6,0.14150,0.12470,0.06213,0.05588,0.2989,0.07380
2,879830,M,17.010,20.26,109.70,904.3,0.08772,0.07304,0.06950,0.05390,...,19.800,25.05,130.00,1210.0,0.11110,0.14860,0.19320,0.10960,0.3275,0.06469
3,8810158,B,13.110,22.54,87.02,529.4,0.10020,0.14830,0.08705,0.05102,...,14.550,29.16,99.48,639.3,0.13490,0.44020,0.31620,0.11260,0.4128,0.10760
4,8810436,B,15.270,12.91,98.17,725.5,0.08182,0.06230,0.05892,0.03157,...,17.380,15.92,113.70,932.7,0.12220,0.21860,0.29620,0.10350,0.2320,0.07474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,926424,M,21.560,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.41070,0.22160,0.2060,0.07115
360,926682,M,20.130,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.32150,0.16280,0.2572,0.06637
361,926954,M,16.600,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.34030,0.14180,0.2218,0.07820
362,927241,M,20.600,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.93870,0.26500,0.4087,0.12400


In [342]:
test_data

,Unnamed: 0,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,0,842302,M,17.990,10.38,122.80,1001.0,0.11840,0.27760,0.30010,...,25.38,17.33,184.60,2019.0,0.16220,0.6656,0.7119,0.26540,0.4601,0.11890
1,1,842517,M,20.570,17.77,132.90,1326.0,0.08474,0.07864,0.08690,...,24.99,23.41,158.80,1956.0,0.12380,0.1866,0.2416,0.18600,0.2750,0.08902
2,2,84300903,M,19.690,21.25,130.00,1203.0,0.10960,0.15990,0.19740,...,23.57,25.53,152.50,1709.0,0.14440,0.4245,0.4504,0.24300,0.3613,0.08758
3,3,84348301,M,11.420,20.38,77.58,386.1,0.14250,0.28390,0.24140,...,14.91,26.50,98.87,567.7,0.20980,0.8663,0.6869,0.25750,0.6638,0.17300
4,4,84358402,M,20.290,14.34,135.10,1297.0,0.10030,0.13280,0.19800,...,22.54,16.67,152.20,1575.0,0.13740,0.2050,0.4000,0.16250,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,109,864018,B,11.340,21.26,72.48,396.5,0.08759,0.06575,0.05133,...,13.01,29.15,83.99,518.1,0.16990,0.2196,0.3120,0.08278,0.2829,0.08832
110,110,864033,B,9.777,16.99,62.50,290.2,0.10370,0.08404,0.04334,...,11.05,21.47,71.68,367.0,0.14670,0.1765,0.1300,0.05334,0.2533,0.08468
111,111,86408,B,12.630,20.76,82.15,480.4,0.09933,0.12090,0.10650,...,13.33,25.47,89.00,527.4,0.12870,0.2250,0.2216,0.11050,0.2226,0.08486
112,112,86409,B,14.260,19.65,97.83,629.9,0.07837,0.22330,0.30030,...,15.30,23.73,107.00,709.0,0.08949,0.4193,0.6783,0.15050,0.2398,0.10820


In [343]:
val_data

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,864496,B,8.726,15.83,55.84,230.9,0.11500,0.08201,0.04132,0.01924,...,9.628,19.62,64.48,284.4,0.1724,0.2364,0.2456,0.10500,0.2926,0.10170
1,864685,B,11.930,21.53,76.53,438.6,0.09768,0.07849,0.03328,0.02008,...,13.670,26.15,87.54,583.0,0.1500,0.2399,0.1503,0.07247,0.2438,0.08541
2,864726,B,8.950,15.76,58.74,245.2,0.09462,0.12430,0.09263,0.02308,...,9.414,17.07,63.34,270.0,0.1179,0.1879,0.1544,0.03846,0.1652,0.07722
3,864729,M,14.870,16.67,98.64,682.5,0.11620,0.16490,0.16900,0.08923,...,18.810,27.37,127.10,1095.0,0.1878,0.4480,0.4704,0.20270,0.3585,0.10650
4,864877,M,15.780,22.91,105.70,782.6,0.11550,0.17520,0.21330,0.09479,...,20.190,30.50,130.30,1272.0,0.1855,0.4925,0.7356,0.20340,0.3274,0.12520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,877501,B,12.230,19.56,78.54,461.0,0.09586,0.08087,0.04187,0.04107,...,14.440,28.36,92.15,638.4,0.1429,0.2042,0.1377,0.10800,0.2668,0.08174
87,877989,M,17.540,19.32,115.10,951.6,0.08968,0.11980,0.10360,0.07488,...,20.420,25.84,139.50,1239.0,0.1381,0.3420,0.3508,0.19390,0.2928,0.07867
88,878796,M,23.290,26.67,158.90,1685.0,0.11410,0.20840,0.35230,0.16200,...,25.120,32.68,177.00,1986.0,0.1536,0.4167,0.7892,0.27330,0.3198,0.08762
89,87880,M,13.810,23.75,91.56,597.8,0.13230,0.17680,0.15580,0.09176,...,19.200,41.85,128.50,1153.0,0.2226,0.5209,0.4646,0.20130,0.4432,0.10860


In [344]:
# Load the testing set
test_data = pd.read_csv(test_path)
print(test_data.shape)
x_test = test_data[test_data.columns[3:]]
print(x_test.shape)
y_test = test_data[test_data.columns[2]]
le = LabelEncoder()
y_test = np.array(le.fit_transform(y_test))
print(le.classes_)

(114, 33)
(114, 30)
['B' 'M']


In [345]:
# Load the validation set
val_data = pd.read_csv(valid_path)
print(val_data.shape)
x_val = val_data[val_data.columns[2:]]
print(x_val.shape)
y_val = val_data[val_data.columns[1]]
le = LabelEncoder()
y_val = np.array(le.fit_transform(y_val))
print(le.classes_)

(91, 32)
(91, 30)
['B' 'M']


In [346]:
x_test

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.990,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.16220,0.6656,0.7119,0.26540,0.4601,0.11890
1,20.570,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.12380,0.1866,0.2416,0.18600,0.2750,0.08902
2,19.690,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.14440,0.4245,0.4504,0.24300,0.3613,0.08758
3,11.420,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.20980,0.8663,0.6869,0.25750,0.6638,0.17300
4,20.290,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.13740,0.2050,0.4000,0.16250,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,11.340,21.26,72.48,396.5,0.08759,0.06575,0.05133,0.01899,0.1487,0.06529,...,13.01,29.15,83.99,518.1,0.16990,0.2196,0.3120,0.08278,0.2829,0.08832
110,9.777,16.99,62.50,290.2,0.10370,0.08404,0.04334,0.01778,0.1584,0.07065,...,11.05,21.47,71.68,367.0,0.14670,0.1765,0.1300,0.05334,0.2533,0.08468
111,12.630,20.76,82.15,480.4,0.09933,0.12090,0.10650,0.06021,0.1735,0.07070,...,13.33,25.47,89.00,527.4,0.12870,0.2250,0.2216,0.11050,0.2226,0.08486
112,14.260,19.65,97.83,629.9,0.07837,0.22330,0.30030,0.07798,0.1704,0.07769,...,15.30,23.73,107.00,709.0,0.08949,0.4193,0.6783,0.15050,0.2398,0.10820


In [347]:
type(x_train)

pandas.core.frame.DataFrame

In [348]:
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()

In [349]:
x_train

array([[1.512e+01, 1.668e+01, 9.878e+01, ..., 1.252e-01, 3.415e-01,
        9.740e-02],
       [9.876e+00, 1.727e+01, 6.292e+01, ..., 5.588e-02, 2.989e-01,
        7.380e-02],
       [1.701e+01, 2.026e+01, 1.097e+02, ..., 1.096e-01, 3.275e-01,
        6.469e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [350]:
x_test

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.263e+01, 2.076e+01, 8.215e+01, ..., 1.105e-01, 2.226e-01,
        8.486e-02],
       [1.426e+01, 1.965e+01, 9.783e+01, ..., 1.505e-01, 2.398e-01,
        1.082e-01],
       [1.051e+01, 2.019e+01, 6.864e+01, ..., 6.136e-02, 2.383e-01,
        9.026e-02]])

In [351]:
x_val

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,8.726,15.83,55.84,230.9,0.11500,0.08201,0.04132,0.01924,0.1649,0.07633,...,9.628,19.62,64.48,284.4,0.1724,0.2364,0.2456,0.10500,0.2926,0.10170
1,11.930,21.53,76.53,438.6,0.09768,0.07849,0.03328,0.02008,0.1688,0.06194,...,13.670,26.15,87.54,583.0,0.1500,0.2399,0.1503,0.07247,0.2438,0.08541
2,8.950,15.76,58.74,245.2,0.09462,0.12430,0.09263,0.02308,0.1305,0.07163,...,9.414,17.07,63.34,270.0,0.1179,0.1879,0.1544,0.03846,0.1652,0.07722
3,14.870,16.67,98.64,682.5,0.11620,0.16490,0.16900,0.08923,0.2157,0.06768,...,18.810,27.37,127.10,1095.0,0.1878,0.4480,0.4704,0.20270,0.3585,0.10650
4,15.780,22.91,105.70,782.6,0.11550,0.17520,0.21330,0.09479,0.2096,0.07331,...,20.190,30.50,130.30,1272.0,0.1855,0.4925,0.7356,0.20340,0.3274,0.12520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,12.230,19.56,78.54,461.0,0.09586,0.08087,0.04187,0.04107,0.1979,0.06013,...,14.440,28.36,92.15,638.4,0.1429,0.2042,0.1377,0.10800,0.2668,0.08174
87,17.540,19.32,115.10,951.6,0.08968,0.11980,0.10360,0.07488,0.1506,0.05491,...,20.420,25.84,139.50,1239.0,0.1381,0.3420,0.3508,0.19390,0.2928,0.07867
88,23.290,26.67,158.90,1685.0,0.11410,0.20840,0.35230,0.16200,0.2200,0.06229,...,25.120,32.68,177.00,1986.0,0.1536,0.4167,0.7892,0.27330,0.3198,0.08762
89,13.810,23.75,91.56,597.8,0.13230,0.17680,0.15580,0.09176,0.2251,0.07421,...,19.200,41.85,128.50,1153.0,0.2226,0.5209,0.4646,0.20130,0.4432,0.10860


In [352]:
type(x_train)

numpy.ndarray

In [353]:
from sklearn.model_selection import train_test_split

print("Shape of training set:", x_train.shape)
print("Shape of test set:", x_test.shape)

Shape of training set: (364, 30)
Shape of test set: (114, 30)


In [354]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV


In [355]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

In [356]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()

parameters = {
    'criterion' : ['gini', 'entropy'],
    'max_depth' : range(2, 32, 1),
    'min_samples_leaf' : range(1, 10, 1),
    'min_samples_split' : range(2, 10, 1),
    'splitter' : ['best', 'random']
}

grid_search_dt = GridSearchCV(dtc, parameters, cv = 5, n_jobs = -1, verbose = 1)
grid_search_dt.fit(x_train, y_train)

Fitting 5 folds for each of 8640 candidates, totalling 43200 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 32),
                         'min_samples_leaf': range(1, 10),
                         'min_samples_split': range(2, 10),
                         'splitter': ['best', 'random']},
             verbose=1)

In [357]:
# best parameters

grid_search_dt.best_params_

{'criterion': 'gini',
 'max_depth': 16,
 'min_samples_leaf': 4,
 'min_samples_split': 8,
 'splitter': 'random'}

In [358]:
grid_search_dt.best_score_

0.9753044140030441

In [359]:
dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = 28, min_samples_leaf = 1, min_samples_split = 8, splitter = 'random')
dtc.fit(x_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=28, min_samples_split=8,
                       splitter='random')

In [360]:
y_pred = dtc.predict(x_test)

In [361]:
# accuracy score

print(accuracy_score(y_train, dtc.predict(x_train)))

dtc_acc = accuracy_score(y_test, dtc.predict(x_test))
print(dtc_acc)

0.9807692307692307
0.8157894736842105


In [362]:
# confusion matrix

print(confusion_matrix(y_test, y_pred))

[[46  0]
 [21 47]]


In [363]:
# classification report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.69      1.00      0.81        46
           1       1.00      0.69      0.82        68

    accuracy                           0.82       114
   macro avg       0.84      0.85      0.82       114
weighted avg       0.87      0.82      0.82       114



In [364]:
# set path split2
split = 'split2'
path = "../5-fold_new/DATA"
train_path = "{}/{}/train/train_{}.csv".format(path,split,split)
test_path =  "{}/test.csv".format(path)
valid_path = "{}/{}/val/val_{}.csv".format(path,split,split)

In [365]:
print(train_path)
print(test_path)
print(valid_path)

../5-fold_new/DATA/split2/train/train_split2.csv
../5-fold_new/DATA/test.csv
../5-fold_new/DATA/split2/val/val_split2.csv


In [366]:
train_data = pd.read_csv(train_path)
print(train_data.shape)
x_train = train_data[train_data.columns[2:]]
print(x_train.shape)
y_train = train_data[train_data.columns[1]]
le = LabelEncoder()
y_train = np.array(le.fit_transform(y_train))
print(le.classes_)

(364, 32)
(364, 30)
['B' 'M']


In [368]:
# Load the validation set
val_data = pd.read_csv(valid_path)
print(val_data.shape)
x_val = val_data[val_data.columns[2:]]
print(x_val.shape)
y_val = val_data[val_data.columns[1]]
le = LabelEncoder()
y_val = np.array(le.fit_transform(y_val))
print(le.classes_)

(91, 32)
(91, 30)
['B' 'M']


In [369]:
train_data

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,864496,B,8.726,15.83,55.84,230.9,0.11500,0.08201,0.04132,0.01924,...,9.628,19.62,64.48,284.4,0.17240,0.23640,0.2456,0.10500,0.2926,0.10170
1,864685,B,11.930,21.53,76.53,438.6,0.09768,0.07849,0.03328,0.02008,...,13.670,26.15,87.54,583.0,0.15000,0.23990,0.1503,0.07247,0.2438,0.08541
2,864726,B,8.950,15.76,58.74,245.2,0.09462,0.12430,0.09263,0.02308,...,9.414,17.07,63.34,270.0,0.11790,0.18790,0.1544,0.03846,0.1652,0.07722
3,864729,M,14.870,16.67,98.64,682.5,0.11620,0.16490,0.16900,0.08923,...,18.810,27.37,127.10,1095.0,0.18780,0.44800,0.4704,0.20270,0.3585,0.10650
4,864877,M,15.780,22.91,105.70,782.6,0.11550,0.17520,0.21330,0.09479,...,20.190,30.50,130.30,1272.0,0.18550,0.49250,0.7356,0.20340,0.3274,0.12520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,926424,M,21.560,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.22160,0.2060,0.07115
360,926682,M,20.130,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.16280,0.2572,0.06637
361,926954,M,16.600,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.14180,0.2218,0.07820
362,927241,M,20.600,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.26500,0.4087,0.12400


In [370]:
test_data

,Unnamed: 0,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,0,842302,M,17.990,10.38,122.80,1001.0,0.11840,0.27760,0.30010,...,25.38,17.33,184.60,2019.0,0.16220,0.6656,0.7119,0.26540,0.4601,0.11890
1,1,842517,M,20.570,17.77,132.90,1326.0,0.08474,0.07864,0.08690,...,24.99,23.41,158.80,1956.0,0.12380,0.1866,0.2416,0.18600,0.2750,0.08902
2,2,84300903,M,19.690,21.25,130.00,1203.0,0.10960,0.15990,0.19740,...,23.57,25.53,152.50,1709.0,0.14440,0.4245,0.4504,0.24300,0.3613,0.08758
3,3,84348301,M,11.420,20.38,77.58,386.1,0.14250,0.28390,0.24140,...,14.91,26.50,98.87,567.7,0.20980,0.8663,0.6869,0.25750,0.6638,0.17300
4,4,84358402,M,20.290,14.34,135.10,1297.0,0.10030,0.13280,0.19800,...,22.54,16.67,152.20,1575.0,0.13740,0.2050,0.4000,0.16250,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,109,864018,B,11.340,21.26,72.48,396.5,0.08759,0.06575,0.05133,...,13.01,29.15,83.99,518.1,0.16990,0.2196,0.3120,0.08278,0.2829,0.08832
110,110,864033,B,9.777,16.99,62.50,290.2,0.10370,0.08404,0.04334,...,11.05,21.47,71.68,367.0,0.14670,0.1765,0.1300,0.05334,0.2533,0.08468
111,111,86408,B,12.630,20.76,82.15,480.4,0.09933,0.12090,0.10650,...,13.33,25.47,89.00,527.4,0.12870,0.2250,0.2216,0.11050,0.2226,0.08486
112,112,86409,B,14.260,19.65,97.83,629.9,0.07837,0.22330,0.30030,...,15.30,23.73,107.00,709.0,0.08949,0.4193,0.6783,0.15050,0.2398,0.10820


In [371]:
val_data

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,879523,M,15.120,16.68,98.78,716.6,0.08876,0.09588,0.07550,0.04079,...,17.77,20.24,117.70,989.5,0.1491,0.3331,0.33270,0.12520,0.3415,0.09740
1,879804,B,9.876,17.27,62.92,295.4,0.10890,0.07232,0.01756,0.01952,...,10.42,23.22,67.08,331.6,0.1415,0.1247,0.06213,0.05588,0.2989,0.07380
2,879830,M,17.010,20.26,109.70,904.3,0.08772,0.07304,0.06950,0.05390,...,19.80,25.05,130.00,1210.0,0.1111,0.1486,0.19320,0.10960,0.3275,0.06469
3,8810158,B,13.110,22.54,87.02,529.4,0.10020,0.14830,0.08705,0.05102,...,14.55,29.16,99.48,639.3,0.1349,0.4402,0.31620,0.11260,0.4128,0.10760
4,8810436,B,15.270,12.91,98.17,725.5,0.08182,0.06230,0.05892,0.03157,...,17.38,15.92,113.70,932.7,0.1222,0.2186,0.29620,0.10350,0.2320,0.07474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,8915,B,14.960,19.10,97.03,687.3,0.08992,0.09823,0.05940,0.04819,...,16.25,26.19,109.10,809.8,0.1313,0.3030,0.18040,0.14890,0.2962,0.08472
87,891670,B,12.950,16.02,83.14,513.7,0.10050,0.07943,0.06155,0.03370,...,13.74,19.93,88.81,585.4,0.1483,0.2068,0.22410,0.10560,0.3380,0.09584
88,891703,B,11.850,17.46,75.54,432.7,0.08372,0.05642,0.02688,0.02280,...,13.06,25.75,84.35,517.8,0.1369,0.1758,0.13160,0.09140,0.3101,0.07007
89,891716,B,12.720,13.78,81.78,492.1,0.09667,0.08393,0.01288,0.01924,...,13.50,17.48,88.54,553.7,0.1298,0.1472,0.05233,0.06343,0.2369,0.06922


In [372]:
# Load the testing set
test_data = pd.read_csv(test_path)
print(test_data.shape)
x_test = test_data[test_data.columns[3:]]
print(x_test.shape)
y_test = test_data[test_data.columns[2]]
le = LabelEncoder()
y_test = np.array(le.fit_transform(y_test))
print(le.classes_)

(114, 33)
(114, 30)
['B' 'M']


In [373]:
# Load the validation set
val_data = pd.read_csv(valid_path)
print(val_data.shape)
x_val = val_data[val_data.columns[2:]]
print(x_val.shape)
y_val = val_data[val_data.columns[1]]
le = LabelEncoder()
y_val = np.array(le.fit_transform(y_val))
print(le.classes_)

(91, 32)
(91, 30)
['B' 'M']


In [374]:
x_test

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.990,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.16220,0.6656,0.7119,0.26540,0.4601,0.11890
1,20.570,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.12380,0.1866,0.2416,0.18600,0.2750,0.08902
2,19.690,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.14440,0.4245,0.4504,0.24300,0.3613,0.08758
3,11.420,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.20980,0.8663,0.6869,0.25750,0.6638,0.17300
4,20.290,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.13740,0.2050,0.4000,0.16250,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,11.340,21.26,72.48,396.5,0.08759,0.06575,0.05133,0.01899,0.1487,0.06529,...,13.01,29.15,83.99,518.1,0.16990,0.2196,0.3120,0.08278,0.2829,0.08832
110,9.777,16.99,62.50,290.2,0.10370,0.08404,0.04334,0.01778,0.1584,0.07065,...,11.05,21.47,71.68,367.0,0.14670,0.1765,0.1300,0.05334,0.2533,0.08468
111,12.630,20.76,82.15,480.4,0.09933,0.12090,0.10650,0.06021,0.1735,0.07070,...,13.33,25.47,89.00,527.4,0.12870,0.2250,0.2216,0.11050,0.2226,0.08486
112,14.260,19.65,97.83,629.9,0.07837,0.22330,0.30030,0.07798,0.1704,0.07769,...,15.30,23.73,107.00,709.0,0.08949,0.4193,0.6783,0.15050,0.2398,0.10820


In [375]:
type(x_train)

pandas.core.frame.DataFrame

In [376]:
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()

In [377]:
x_train

array([[8.726e+00, 1.583e+01, 5.584e+01, ..., 1.050e-01, 2.926e-01,
        1.017e-01],
       [1.193e+01, 2.153e+01, 7.653e+01, ..., 7.247e-02, 2.438e-01,
        8.541e-02],
       [8.950e+00, 1.576e+01, 5.874e+01, ..., 3.846e-02, 1.652e-01,
        7.722e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [378]:
x_test

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.263e+01, 2.076e+01, 8.215e+01, ..., 1.105e-01, 2.226e-01,
        8.486e-02],
       [1.426e+01, 1.965e+01, 9.783e+01, ..., 1.505e-01, 2.398e-01,
        1.082e-01],
       [1.051e+01, 2.019e+01, 6.864e+01, ..., 6.136e-02, 2.383e-01,
        9.026e-02]])

In [379]:
x_val

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,15.120,16.68,98.78,716.6,0.08876,0.09588,0.07550,0.04079,0.1594,0.05986,...,17.77,20.24,117.70,989.5,0.1491,0.3331,0.33270,0.12520,0.3415,0.09740
1,9.876,17.27,62.92,295.4,0.10890,0.07232,0.01756,0.01952,0.1934,0.06285,...,10.42,23.22,67.08,331.6,0.1415,0.1247,0.06213,0.05588,0.2989,0.07380
2,17.010,20.26,109.70,904.3,0.08772,0.07304,0.06950,0.05390,0.2026,0.05223,...,19.80,25.05,130.00,1210.0,0.1111,0.1486,0.19320,0.10960,0.3275,0.06469
3,13.110,22.54,87.02,529.4,0.10020,0.14830,0.08705,0.05102,0.1850,0.07310,...,14.55,29.16,99.48,639.3,0.1349,0.4402,0.31620,0.11260,0.4128,0.10760
4,15.270,12.91,98.17,725.5,0.08182,0.06230,0.05892,0.03157,0.1359,0.05526,...,17.38,15.92,113.70,932.7,0.1222,0.2186,0.29620,0.10350,0.2320,0.07474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,14.960,19.10,97.03,687.3,0.08992,0.09823,0.05940,0.04819,0.1879,0.05852,...,16.25,26.19,109.10,809.8,0.1313,0.3030,0.18040,0.14890,0.2962,0.08472
87,12.950,16.02,83.14,513.7,0.10050,0.07943,0.06155,0.03370,0.1730,0.06470,...,13.74,19.93,88.81,585.4,0.1483,0.2068,0.22410,0.10560,0.3380,0.09584
88,11.850,17.46,75.54,432.7,0.08372,0.05642,0.02688,0.02280,0.1875,0.05715,...,13.06,25.75,84.35,517.8,0.1369,0.1758,0.13160,0.09140,0.3101,0.07007
89,12.720,13.78,81.78,492.1,0.09667,0.08393,0.01288,0.01924,0.1638,0.06100,...,13.50,17.48,88.54,553.7,0.1298,0.1472,0.05233,0.06343,0.2369,0.06922


In [380]:
type(x_train)

numpy.ndarray

In [381]:
from sklearn.model_selection import train_test_split

print("Shape of training set:", x_train.shape)
print("Shape of test set:", x_test.shape)

Shape of training set: (364, 30)
Shape of test set: (114, 30)


In [382]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV


In [383]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

In [384]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()

parameters = {
    'criterion' : ['gini', 'entropy'],
    'max_depth' : range(2, 32, 1),
    'min_samples_leaf' : range(1, 10, 1),
    'min_samples_split' : range(2, 10, 1),
    'splitter' : ['best', 'random']
}

grid_search_dt = GridSearchCV(dtc, parameters, cv = 5, n_jobs = -1, verbose = 1)
grid_search_dt.fit(x_train, y_train)

Fitting 5 folds for each of 8640 candidates, totalling 43200 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 32),
                         'min_samples_leaf': range(1, 10),
                         'min_samples_split': range(2, 10),
                         'splitter': ['best', 'random']},
             verbose=1)

In [385]:
# best parameters

grid_search_dt.best_params_

{'criterion': 'gini',
 'max_depth': 8,
 'min_samples_leaf': 8,
 'min_samples_split': 3,
 'splitter': 'random'}

In [386]:
grid_search_dt.best_score_

0.9725266362252662

In [387]:
dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = 28, min_samples_leaf = 1, min_samples_split = 8, splitter = 'random')
dtc.fit(x_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=28, min_samples_split=8,
                       splitter='random')

In [388]:
y_pred = dtc.predict(x_test)
y_pred

array([1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0])

In [389]:
# accuracy score

print(accuracy_score(y_train, dtc.predict(x_train)))

dtc_acc = accuracy_score(y_test, dtc.predict(x_test))
print(dtc_acc)

0.989010989010989
0.8157894736842105


In [390]:
# confusion matrix

print(confusion_matrix(y_test, y_pred))

[[46  0]
 [21 47]]


In [391]:
# classification report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.69      1.00      0.81        46
           1       1.00      0.69      0.82        68

    accuracy                           0.82       114
   macro avg       0.84      0.85      0.82       114
weighted avg       0.87      0.82      0.82       114



In [392]:
# set path split3
split = 'split3'
path = "../5-fold_new/DATA"
train_path = "{}/{}/train/train_{}.csv".format(path,split,split)
test_path =  "{}/test.csv".format(path)
valid_path = "{}/{}/val/val_{}.csv".format(path,split,split)

In [393]:
print(train_path)
print(test_path)
print(valid_path)

../5-fold_new/DATA/split3/train/train_split3.csv
../5-fold_new/DATA/test.csv
../5-fold_new/DATA/split3/val/val_split3.csv


In [394]:
train_data = pd.read_csv(train_path)
print(train_data.shape)
x_train = train_data[train_data.columns[2:]]
print(x_train.shape)
y_train = train_data[train_data.columns[1]]
le = LabelEncoder()
y_train = np.array(le.fit_transform(y_train))
print(le.classes_)

(364, 32)
(364, 30)
['B' 'M']


In [395]:
# Load the validation set
val_data = pd.read_csv(valid_path)
print(val_data.shape)
x_val = val_data[val_data.columns[2:]]
print(x_val.shape)
y_val = val_data[val_data.columns[1]]
le = LabelEncoder()
y_val = np.array(le.fit_transform(y_val))
print(le.classes_)

(91, 32)
(91, 30)
['B' 'M']


In [396]:
train_data

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,864496,B,8.726,15.83,55.84,230.9,0.11500,0.08201,0.04132,0.01924,...,9.628,19.62,64.48,284.4,0.17240,0.23640,0.2456,0.10500,0.2926,0.10170
1,864685,B,11.930,21.53,76.53,438.6,0.09768,0.07849,0.03328,0.02008,...,13.670,26.15,87.54,583.0,0.15000,0.23990,0.1503,0.07247,0.2438,0.08541
2,864726,B,8.950,15.76,58.74,245.2,0.09462,0.12430,0.09263,0.02308,...,9.414,17.07,63.34,270.0,0.11790,0.18790,0.1544,0.03846,0.1652,0.07722
3,864729,M,14.870,16.67,98.64,682.5,0.11620,0.16490,0.16900,0.08923,...,18.810,27.37,127.10,1095.0,0.18780,0.44800,0.4704,0.20270,0.3585,0.10650
4,864877,M,15.780,22.91,105.70,782.6,0.11550,0.17520,0.21330,0.09479,...,20.190,30.50,130.30,1272.0,0.18550,0.49250,0.7356,0.20340,0.3274,0.12520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,926424,M,21.560,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.22160,0.2060,0.07115
360,926682,M,20.130,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.16280,0.2572,0.06637
361,926954,M,16.600,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.14180,0.2218,0.07820
362,927241,M,20.600,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.26500,0.4087,0.12400


In [397]:
test_data

,Unnamed: 0,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,0,842302,M,17.990,10.38,122.80,1001.0,0.11840,0.27760,0.30010,...,25.38,17.33,184.60,2019.0,0.16220,0.6656,0.7119,0.26540,0.4601,0.11890
1,1,842517,M,20.570,17.77,132.90,1326.0,0.08474,0.07864,0.08690,...,24.99,23.41,158.80,1956.0,0.12380,0.1866,0.2416,0.18600,0.2750,0.08902
2,2,84300903,M,19.690,21.25,130.00,1203.0,0.10960,0.15990,0.19740,...,23.57,25.53,152.50,1709.0,0.14440,0.4245,0.4504,0.24300,0.3613,0.08758
3,3,84348301,M,11.420,20.38,77.58,386.1,0.14250,0.28390,0.24140,...,14.91,26.50,98.87,567.7,0.20980,0.8663,0.6869,0.25750,0.6638,0.17300
4,4,84358402,M,20.290,14.34,135.10,1297.0,0.10030,0.13280,0.19800,...,22.54,16.67,152.20,1575.0,0.13740,0.2050,0.4000,0.16250,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,109,864018,B,11.340,21.26,72.48,396.5,0.08759,0.06575,0.05133,...,13.01,29.15,83.99,518.1,0.16990,0.2196,0.3120,0.08278,0.2829,0.08832
110,110,864033,B,9.777,16.99,62.50,290.2,0.10370,0.08404,0.04334,...,11.05,21.47,71.68,367.0,0.14670,0.1765,0.1300,0.05334,0.2533,0.08468
111,111,86408,B,12.630,20.76,82.15,480.4,0.09933,0.12090,0.10650,...,13.33,25.47,89.00,527.4,0.12870,0.2250,0.2216,0.11050,0.2226,0.08486
112,112,86409,B,14.260,19.65,97.83,629.9,0.07837,0.22330,0.30030,...,15.30,23.73,107.00,709.0,0.08949,0.4193,0.6783,0.15050,0.2398,0.10820


In [398]:
val_data

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,891936,B,10.91,12.35,69.14,363.7,0.08518,0.04721,0.01236,0.01369,...,11.37,14.82,72.42,392.2,0.09312,0.07506,0.02884,0.03194,0.2143,0.06643
1,892189,M,11.76,18.14,75.00,431.1,0.09968,0.05914,0.02685,0.03515,...,13.36,23.39,85.10,553.6,0.11370,0.07974,0.06120,0.07160,0.1978,0.06915
2,892214,B,14.26,18.17,91.22,633.1,0.06576,0.05220,0.02475,0.01374,...,16.22,25.26,105.80,819.7,0.09445,0.21670,0.15650,0.07530,0.2636,0.07676
3,892399,B,10.51,23.09,66.85,334.2,0.10150,0.06797,0.02495,0.01875,...,10.93,24.22,70.10,362.7,0.11430,0.08614,0.04158,0.03125,0.2227,0.06777
4,892438,M,19.53,18.90,129.50,1217.0,0.11500,0.16420,0.21970,0.10620,...,25.93,26.24,171.10,2053.0,0.14950,0.41160,0.61210,0.19800,0.2968,0.09929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,90250,B,12.05,22.72,78.75,447.8,0.06935,0.10730,0.07943,0.02978,...,12.57,28.71,87.36,488.4,0.08799,0.32140,0.29120,0.10920,0.2191,0.09349
87,90251,B,12.39,17.48,80.64,462.9,0.10420,0.12970,0.05892,0.02880,...,14.18,23.13,95.23,600.5,0.14270,0.35930,0.32060,0.09804,0.2819,0.11180
88,902727,B,13.28,13.72,85.79,541.8,0.08363,0.08575,0.05077,0.02864,...,14.24,17.37,96.59,623.7,0.11660,0.26850,0.28660,0.09173,0.2736,0.07320
89,90291,M,14.60,23.29,93.97,664.7,0.08682,0.06636,0.08390,0.05271,...,15.79,31.71,102.20,758.2,0.13120,0.15810,0.26750,0.13590,0.2477,0.06836


In [399]:
# Load the testing set
test_data = pd.read_csv(test_path)
print(test_data.shape)
x_test = test_data[test_data.columns[3:]]
print(x_test.shape)
y_test = test_data[test_data.columns[2]]
le = LabelEncoder()
y_test = np.array(le.fit_transform(y_test))
print(le.classes_)

(114, 33)
(114, 30)
['B' 'M']


In [400]:
# Load the validation set
val_data = pd.read_csv(valid_path)
print(val_data.shape)
x_val = val_data[val_data.columns[2:]]
print(x_val.shape)
y_val = val_data[val_data.columns[1]]
le = LabelEncoder()
y_val = np.array(le.fit_transform(y_val))
print(le.classes_)

(91, 32)
(91, 30)
['B' 'M']


In [401]:
x_test

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.990,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.16220,0.6656,0.7119,0.26540,0.4601,0.11890
1,20.570,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.12380,0.1866,0.2416,0.18600,0.2750,0.08902
2,19.690,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.14440,0.4245,0.4504,0.24300,0.3613,0.08758
3,11.420,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.20980,0.8663,0.6869,0.25750,0.6638,0.17300
4,20.290,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.13740,0.2050,0.4000,0.16250,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,11.340,21.26,72.48,396.5,0.08759,0.06575,0.05133,0.01899,0.1487,0.06529,...,13.01,29.15,83.99,518.1,0.16990,0.2196,0.3120,0.08278,0.2829,0.08832
110,9.777,16.99,62.50,290.2,0.10370,0.08404,0.04334,0.01778,0.1584,0.07065,...,11.05,21.47,71.68,367.0,0.14670,0.1765,0.1300,0.05334,0.2533,0.08468
111,12.630,20.76,82.15,480.4,0.09933,0.12090,0.10650,0.06021,0.1735,0.07070,...,13.33,25.47,89.00,527.4,0.12870,0.2250,0.2216,0.11050,0.2226,0.08486
112,14.260,19.65,97.83,629.9,0.07837,0.22330,0.30030,0.07798,0.1704,0.07769,...,15.30,23.73,107.00,709.0,0.08949,0.4193,0.6783,0.15050,0.2398,0.10820


In [402]:
type(x_train)

pandas.core.frame.DataFrame

In [403]:
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()

In [404]:
x_train

array([[8.726e+00, 1.583e+01, 5.584e+01, ..., 1.050e-01, 2.926e-01,
        1.017e-01],
       [1.193e+01, 2.153e+01, 7.653e+01, ..., 7.247e-02, 2.438e-01,
        8.541e-02],
       [8.950e+00, 1.576e+01, 5.874e+01, ..., 3.846e-02, 1.652e-01,
        7.722e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [405]:
x_test

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.263e+01, 2.076e+01, 8.215e+01, ..., 1.105e-01, 2.226e-01,
        8.486e-02],
       [1.426e+01, 1.965e+01, 9.783e+01, ..., 1.505e-01, 2.398e-01,
        1.082e-01],
       [1.051e+01, 2.019e+01, 6.864e+01, ..., 6.136e-02, 2.383e-01,
        9.026e-02]])

In [406]:
x_val

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,10.91,12.35,69.14,363.7,0.08518,0.04721,0.01236,0.01369,0.1449,0.06031,...,11.37,14.82,72.42,392.2,0.09312,0.07506,0.02884,0.03194,0.2143,0.06643
1,11.76,18.14,75.00,431.1,0.09968,0.05914,0.02685,0.03515,0.1619,0.06287,...,13.36,23.39,85.10,553.6,0.11370,0.07974,0.06120,0.07160,0.1978,0.06915
2,14.26,18.17,91.22,633.1,0.06576,0.05220,0.02475,0.01374,0.1635,0.05586,...,16.22,25.26,105.80,819.7,0.09445,0.21670,0.15650,0.07530,0.2636,0.07676
3,10.51,23.09,66.85,334.2,0.10150,0.06797,0.02495,0.01875,0.1695,0.06556,...,10.93,24.22,70.10,362.7,0.11430,0.08614,0.04158,0.03125,0.2227,0.06777
4,19.53,18.90,129.50,1217.0,0.11500,0.16420,0.21970,0.10620,0.1792,0.06552,...,25.93,26.24,171.10,2053.0,0.14950,0.41160,0.61210,0.19800,0.2968,0.09929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,12.05,22.72,78.75,447.8,0.06935,0.10730,0.07943,0.02978,0.1203,0.06659,...,12.57,28.71,87.36,488.4,0.08799,0.32140,0.29120,0.10920,0.2191,0.09349
87,12.39,17.48,80.64,462.9,0.10420,0.12970,0.05892,0.02880,0.1779,0.06588,...,14.18,23.13,95.23,600.5,0.14270,0.35930,0.32060,0.09804,0.2819,0.11180
88,13.28,13.72,85.79,541.8,0.08363,0.08575,0.05077,0.02864,0.1617,0.05594,...,14.24,17.37,96.59,623.7,0.11660,0.26850,0.28660,0.09173,0.2736,0.07320
89,14.60,23.29,93.97,664.7,0.08682,0.06636,0.08390,0.05271,0.1627,0.05416,...,15.79,31.71,102.20,758.2,0.13120,0.15810,0.26750,0.13590,0.2477,0.06836


In [407]:
type(x_train)

numpy.ndarray

In [408]:
from sklearn.model_selection import train_test_split

print("Shape of training set:", x_train.shape)
print("Shape of test set:", x_test.shape)

Shape of training set: (364, 30)
Shape of test set: (114, 30)


In [409]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV


In [410]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

In [411]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()

parameters = {
    'criterion' : ['gini', 'entropy'],
    'max_depth' : range(2, 32, 1),
    'min_samples_leaf' : range(1, 10, 1),
    'min_samples_split' : range(2, 10, 1),
    'splitter' : ['best', 'random']
}

grid_search_dt = GridSearchCV(dtc, parameters, cv = 5, n_jobs = -1, verbose = 1)
grid_search_dt.fit(x_train, y_train)

Fitting 5 folds for each of 8640 candidates, totalling 43200 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 32),
                         'min_samples_leaf': range(1, 10),
                         'min_samples_split': range(2, 10),
                         'splitter': ['best', 'random']},
             verbose=1)

In [412]:
# best parameters

grid_search_dt.best_params_

{'criterion': 'gini',
 'max_depth': 21,
 'min_samples_leaf': 2,
 'min_samples_split': 7,
 'splitter': 'random'}

In [413]:
grid_search_dt.best_score_

0.9752663622526635

In [414]:
dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = 28, min_samples_leaf = 1, min_samples_split = 8, splitter = 'random')
dtc.fit(x_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=28, min_samples_split=8,
                       splitter='random')

In [415]:
y_pred = dtc.predict(x_test)
y_pred

array([1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0])

In [416]:
# accuracy score

print(accuracy_score(y_train, dtc.predict(x_train)))

dtc_acc = accuracy_score(y_test, dtc.predict(x_test))
print(dtc_acc)

0.9862637362637363
0.8157894736842105


In [417]:
# confusion matrix

print(confusion_matrix(y_test, y_pred))

[[46  0]
 [21 47]]


In [418]:
# classification report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.69      1.00      0.81        46
           1       1.00      0.69      0.82        68

    accuracy                           0.82       114
   macro avg       0.84      0.85      0.82       114
weighted avg       0.87      0.82      0.82       114

